# Auror Location Problem

The wizarding world has been divided into eight districts.  
The time (in seconds) required to travel from one district to another
via the floo network is shown below:  
$$\begin{array}{9c}
  & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 \\
1 &   & 3 & 4 & 6 & 8 & 9 & 8 &10 \\
2 & 3 &   & 5 & 4 & 8 & 6 &12 & 9 \\
3 & 4 & 5 &   & 2 & 2 & 3 & 5 & 7 \\
4 & 6 & 4 & 2 &   & 3 & 2 & 5 & 4 \\
5 & 8 & 8 & 2 & 3 &   & 2 & 2 & 4 \\
6 & 9 & 6 & 3 & 2 & 2 &   & 3 & 2 \\
7 & 8 &12 & 5 & 5 & 2 & 3 &   & 2 \\
8 &10 & 9 & 7 & 4 & 4 & 2 & 2 &
\end{array}$$
The wizard population of each district (in thousands) is:
district 1, 40; 2, 30; 3, 35; 4, 20; 5, 15; 6, 50; 7, 45; 8, 60.
The Ministry of Magic has declared that there will be $n$ auror
locations.  

### Determine the locations of the aurors that maximize the number of people who live within two seconds of an auror.  Do this for $n=1,2,3,4$.

In [1]:
# Load the gams extension
%load_ext gams_magic

In [3]:
%%gams
set district /1*8/;
alias(district,d,dd)

parameter population /1 40, 2 30, 3 35, 4 20, 5 15, 6 50, 7 45, 8 60/;

table data(d,dd)
    1  2  3  4  5  6  7  8
1      3  4  6  8  9  8 10
2   3     5  4  8  6 12  9
3   4  5     2  2  3  5  7
4   6  4  2     3  2  5  4
5   8  8  2  3     2  2  4
6   9  6  3  2  2     3  2
7   8 12  5  5  2  3     2
8  10  9  7  4  4  2  2   ;

scalar n /1/;

set zone(d,dd);

zone(d,dd) = yes$(data(d,dd) <= 2);

binary variable build(district);
binary variable x(district);
free variable z;

equations neq, xeq, obj;
xeq(d).. x(d) =e= sum(dd, build(dd)$(zone(d,dd)));
neq.. n =e= sum(d, build(d));
obj.. z =e= sum(d, x(d)*population(d));

model auror /all/;
solve auror using mip maximizing z;
display build.L;

parameter toBuild(*, district), people(*);

toBuild('1', district) = build.L(district);
people('1') = z.L;

n = 2;
solve auror using mip maximizing z;
toBuild('2', district) = build.L(district);
people('2') = z.L;

n = 3;
solve auror using mip maximizing z;
toBuild('3', district) = build.L(district);
people('3') = z.L;

n = 4;
solve auror using mip maximizing z;
toBuild('4', district) = build.L(district);
people('4') = z.L;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),155.0,10,17,MIP,CPLEX,0
1,Normal (1),Optimal Global (1),225.0,10,17,MIP,CPLEX,0.016
2,Normal (1),Optimal Global (1),265.0,10,17,MIP,CPLEX,0
3,Normal (1),Optimal Global (1),295.0,10,17,MIP,CPLEX,0


In [4]:
# parameter tobuild(*,district), people(*);
%gams_pull -d tobuild 
%gams_pull people
for n in range(len(people)):
    rows = tobuild['*'] == str(n+1)
    print(n+1,"(",people[n][1],"):",tobuild.loc[rows, 'district'].tolist())

1 ( 155.0 ): ['8']
2 ( 225.0 ): ['3', '8']
3 ( 265.0 ): ['1', '3', '8']
4 ( 295.0 ): ['1', '2', '4', '7']
